### References Langchain
https://python.langchain.com/v0.1/docs/use_cases/question_answering/quickstart/

### References Chroma db  
https://docs.trychroma.com/guides  
https://github.com/neo-con/chromadb-tutorial  
https://python.langchain.com/v0.1/docs/integrations/vectorstores/chroma/  


In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
import os
import sys
sys.path.append('../..')

from py3810.myUtils import pickle_dump, pickle_load
path_lumen_docs = '..\langchain\docs\lumen\\docs\\'
path_chatbot = '..\langchain\chatbot\\'

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('.env\my_api_key.env')) # read local .env file

os.environ["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"]
os.environ["SECRET_KEY"]


In [ ]:
import textwrap

def print_wrapped(text, width=80):
  """
  Prints a long string to the console, wrapped to fit within a specified width.

  Args:
      text: The long string to be wrapped.
      width: The desired width for each line (default: 80 columns).
  """
  wrapped_text = textwrap.wrap(text, width=width)
  for line in wrapped_text:
    print(line)

# Example usage
long_string = "This is a very long string that needs to be wrapped to fit within 80 columns. It can contain spaces, punctuation, and even newlines."
print_wrapped(long_string)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
persist_directory = path_chatbot + "./chroma_openai_ef"
embedding_function = OpenAIEmbeddings()
collection_name = 'lumen_docs_combined'

#### Only run cell below to add new data


In [ ]:
# # load data
# docs = pickle_load(filename_pickle='lumen_docs_combined', path_pickle_dump=path_chatbot)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)

# # save to disk
# db0 = Chroma.from_documents(
#   documents=splits,
#   embedding=embedding_function,
#   collection_name=collection_name,
#   persist_directory=persist_directory
#   )
# db0_ans = db0.similarity_search(query)
# # print(docs[0].page_content)
# print(f'db0_ans:{db0_ans}')

In [ ]:
my_query = 'where is lumen optometric'

# load from disk
vectorstore = Chroma(
  embedding_function=embedding_function,
  collection_name=collection_name,
  persist_directory=persist_directory
  )
ans = vectorstore.similarity_search(my_query)
print(f'ans:{ans}')

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
queries = [
  "What is lumen optometric's phone number?",
  "What is lumen optometric's address?",
  "What is lumen optometric's location?",
  "do you take vision insurance",
  "what type of insurance do you take",
  "what are the names of the insurance that you take",
  "What is Ortho-k",
  "What does research say about Ortho-k",
  "What equipment do they have?",
  "What equipment do you use for othro-k?",
  ]

In [ ]:
# for query in queries[0:2]:
for query in queries[0:3]:
  print(f'query: {query}')
  print_wrapped(f'answr: {rag_chain.invoke(query)}')
  print('')
  print_wrapped(f'relevant docs: {retriever.get_relevant_documents(query)}')
  print(f'{"="*5}\n')